In [73]:
import numpy as np
import pandas as pd

In [74]:
df = pd.read_csv(r"D:\Python\Kaggle_Datasets\AmazonReview_Analysis\Datasets\amazon_reviews.csv")

In [75]:
df.head()

,Text,label
0,This is the best apps acording to a bunch of ...,1
1,This is a pretty good version of the game for ...,1
2,this is a really . there are a bunch of levels...,1
3,"This is a silly game and can be frustrating, b...",1
4,This is a terrific game on any pad. Hrs of fun...,1


In [76]:
df.isnull().sum()

Text     0
label    0
dtype: int64

In [77]:
df.duplicated().value_counts()

False    19996
Name: count, dtype: int64

In [78]:
df.shape

(19996, 2)

In [79]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")
stops = set(stopwords.words('english'))
stops.remove('not')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bhati\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bhati\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\bhati\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [80]:
df['Text'] = df['Text'].apply(lambda x:x.lower())

In [81]:
def remove_stopwords(text):
    words = word_tokenize(text)  # Tokenize text
    return " ".join([word for word in words if word.lower() not in stops])  # ✅ Use stop_words

# Apply stopword removal
df["Text"] = df["Text"].apply(remove_stopwords)

In [82]:
df['Text'] = df['Text'].apply(word_tokenize)

In [83]:
df['Text'] = df['Text'].apply(lambda x:" ".join(x))

In [84]:
X = df['Text']
y = df['label']

In [85]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25,random_state=42)

In [86]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features=6000)

In [87]:
X_train =tfidf.fit_transform(X_train).toarray()
X_test = tfidf.transform(X_test).toarray()

In [88]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report,ConfusionMatrixDisplay, \
                            precision_score, recall_score, f1_score, roc_auc_score,roc_curve 

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Train the Random Forest Classifier
rfc = RandomForestClassifier(n_estimators=100, random_state=42)
rfc.fit(X_train, y_train)

# Evaluate model
y_pred = rfc.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.4f}")



Model Accuracy: 0.8768


In [94]:
import pickle
import os

# Ensure "models" directory exists
os.makedirs("models", exist_ok=True)

# Save model and vectorizer
with open(r"models\sentiment_model.pkl", "wb") as model_file:
    pickle.dump(rfc, model_file)

with open(r"models\tfidf_vectorizer.pkl", "wb") as vectorizer_file:
    pickle.dump(tfidf, vectorizer_file)

print("Model and vectorizer saved successfully!")

Model and vectorizer saved successfully!
